# Create a Gender_Rank column to apply the correct gender rank (M, F) for filtering based on gender

In [202]:
import pandas as pd
import glob
import os

### The following can be used for batch importing and concatenation. Used during initial joing of raw files.

In [5]:
# Set the folder path
folder_path = '../data/raw/utmb_format'

# Find all CSV files in the folder
csv_files_utmb_format = glob.glob(os.path.join(folder_path, '*.csv'))

# Read and combine all CSVs
df_list = []
for file in csv_files_utmb_format:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# Combine into one DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

### Import a singular raw file that needs to be concatenated with the combined master dataset

In [294]:
combined_df = pd.read_csv("../data/raw/utmb_format/festival_des_templiers_100K_df_raw.csv", encoding = 'utf-8')

In [295]:
combined_df.head()

,Series_ID,Race_ID,Race_Date,Year,Rank,Status,Name,Nationality,Gender,Age_Category,Time,Race_Name,Race_Loc,Race_Dist
0,623,62321,2021-10-22,2021,1,Finisher,vincent viet,FRA,M,35-39,10:52:25,Festival des Templiers 100K,"Millau, France",100K
1,623,62321,2021-10-22,2021,2,Finisher,aurelien collet,FRA,M,40-44,11:03:54,Festival des Templiers 100K,"Millau, France",100K
2,623,62321,2021-10-22,2021,3,Finisher,clement desille,FRA,M,20-34,11:28:34,Festival des Templiers 100K,"Millau, France",100K
3,623,62321,2021-10-22,2021,4,Finisher,matthieu durand,FRA,M,20-34,11:30:12,Festival des Templiers 100K,"Millau, France",100K
4,623,62321,2021-10-22,2021,5,Finisher,romain olivier,FRA,M,35-39,11:30:18,Festival des Templiers 100K,"Millau, France",100K


### Temporarily remove DNFs, sort_values, assign Gender_Rank, merge back to the original dataset

In [296]:
# Filter out DNFs only
df_finished = combined_df[combined_df['Status'].str.upper() != 'DNF'].copy()

# Sort by Race_ID, Gender, and Time
df_finished = df_finished.sort_values(by=['Race_ID', 'Gender', 'Time'])

# Assign Gender_Rank
df_finished['Gender_Rank'] = df_finished.groupby(['Race_ID', 'Gender']).cumcount() + 1

# Step 4: Merge back using Race_ID and Name
df = combined_df.merge(
    df_finished[['Race_ID', 'Name', 'Gender', 'Time', 'Gender_Rank']],
    on=['Race_ID', 'Name', 'Gender', 'Time'],
    how='left'
)

In [297]:
df.shape

(5942, 15)

### Combined master dataset uses 3 letter codes for countries. The following is a dictionary to map countries to their 3 letter code.

In [298]:
country_to_code = {
    'USA': 'USA',
    'POLAND': 'POL',
    'FRA': 'FRA',
    'FRANCE': 'FRA',
    'UNITED KINGDOM': 'GBR',
    'ICELAND': 'ISL',
    'SPAIN': 'ESP',
    'SWITZERLAND': 'CHE',
    'IRELAND': 'IRL',
    'ITALY': 'ITA',
    'LATVIA': 'LVA',
    'SOUTH AFRICA': 'ZAF',
    'MOROCCO': 'MAR',
    'LUXEMBOURG': 'LUX',
    'CHINA': 'CHN',
    'VENEZUELA': 'VEN',
    'JAPAN': 'JPN',
    'PORTUGAL': 'PRT',
    'MAURITIUS': 'MUS',
    'NEW ZEALAND': 'NZL',
    'SWEDEN': 'SWE',
    'ARGENTINA': 'ARG',
    'SLOVAKIA': 'SVK',
    'GERMANY': 'DEU',
    'ALGERIA': 'DZA',
    'AUSTRALIA': 'AUS',
    'CAMBODIA': 'KHM',
    'NORWAY': 'NOR',
    'CROATIA': 'HRV',
    'MEXICO': 'MEX',
    'COLOMBIA': 'COL',
    'CANADA': 'CAN',
    'NETHERLANDS': 'NLD',
    'BRAZIL': 'BRA',
    'TÜRKIYE': 'TUR',
    'BELGIUM': 'BEL',
    'CHILE': 'CHL',
    'SOUTH KOREA': 'KOR',
    'DENMARK': 'DNK',
    'RUSSIA': 'RUS',
    'SLOVENIA': 'SVN',
    'UKRAINE': 'UKR',
    'SERBIA': 'SRB',
    'HUNGARY': 'HUN',
    'ROMANIA': 'ROU',
    'URUGUAY': 'URY',
    'ANDORRA': 'AND',
    'ECUADOR': 'ECU',
    'FINLAND': 'FIN',
    'HONG KONG, CHINA': 'HKG',
    'BELARUS': 'BLR',
    'SINGAPORE': 'SGP',
    'CZECH REPUBLIC': 'CZE',
    'BULGARIA': 'BGR',
    'PERU': 'PER',
    'GREECE': 'GRC',
    'PHILIPPINES': 'PHL',
    'LITHUANIA': 'LTU',
    'MALTA': 'MLT',
    'BOSNIA AND HERZEGOVINA': 'BIH',
    'THAILAND': 'THA',
    'AUSTRIA': 'AUT',
    'ESTONIA': 'EST',
    'CHINESE TAIPEI': 'TWN',
    'INDONESIA': 'IDN',
    'VIETNAM': 'VNM',
    'INDIA': 'IND',
    'MALAYSIA': 'MYS',
    'COSTA RICA': 'CRI',
    'ISRAEL': 'ISR',
    'SAUDI ARABIA': 'SAU',
    'NIGER': 'NER',
    'MOLDOVA': 'MDA',
    'EGYPT': 'EGY',
    'MONTENEGRO': 'MNE',
    'KOSOVO': 'XKX',
    'IRAN': 'IRN',
    'EL SALVADOR': 'SLV',
    'LEBANON': 'LBN',
    'PARAGUAY': 'PRY',
    'PUERTO RICO': 'PRI',
    'OMAN': 'OMN',
    'ALBANIA': 'ALB',
    'BENIN': 'BEN',
    'NORTH MACEDONIA (REPUBLIC OF)': 'MKD',
    'TUNISIA': 'TUN',
    'NEPAL': 'NPL',
    'QATAR': 'QAT',
    'DOMINICAN REPUBLIC': 'DOM',
    'SAN MARINO': 'SMR',
    'NICARAGUA': 'NIC',
    'GUATEMALA': 'GTM',
    'GAMBIA': 'GMB',
    'MONACO': 'MCO',
    'KYRGYZSTAN': 'KGZ',
    'KENYA': 'KEN',
    'BOLIVIA': 'BOL',
    'BAHRAIN': 'BHR',
    'MACAO, CHINA': 'MAC',
    'TOGO': 'TGO',
    'CYPRUS': 'CYP',
    'KUWAIT': 'KWT',
    'NAMIBIA': 'NAM',
    'ZIMBABWE': 'ZWE',
    'UGANDA': 'UGA',
    'LIECHTENSTEIN': 'LIE',
    'HAITI': 'HTI',
    'SWAZILAND': 'SWZ',
    'MONGOLIA': 'MNG',
    'CAMEROON': 'CMR',
    'KAZAKHSTAN': 'KAZ',
    'NEUTRAL ATHLETE': None,
    'UNITED ARAB EMIRATES': 'ARE',
    'MADAGASCAR': 'MDG',
    'AFGHANISTAN': 'AFG',
    'HONDURAS': 'HND',
    'GEORGIA': 'GEO',
    'JORDAN': 'JOR',
    'AZERBAIJAN': 'AZE',
    'BANGLADESH': 'BGD',
    'SENEGAL': 'SEN',
    'BRUNEI': 'BRN',
    'KINGDOM': None,
    'ZEALAND': None,
    'KOREA': None,
    'AMERICA': None,
    'TAIPEI': None,
    'AFRICA': None,
    'ATHLETE': None,
    'SAO TOME AND PRINCIPE': 'STP',
    'PANAMA': 'PAN',
    'CONGO': 'COG',
    'JAMAICA': 'JAM',
    'FRENCH POLYNESIA': 'PYF',
    'IRAQ': 'IRQ',
    'UZBEKISTAN': 'UZB',
    'PAKISTAN': 'PAK',
    'NORTHERN MARIANA ISLANDS': 'MNP',
    'SOUTH SUDAN': 'SSD',
    'ARMENIA': 'ARM',
    'BOTSWANA': 'BWA',
    'BAHAMAS': 'BHS',
    'NORTH KOREA': 'PRK',
    'VANUATU': 'VUT'
}

In [299]:
df['Nationality'] = df['Nationality'].map(country_to_code_cleaned)

In [302]:
df.tail()

(5942, 15)

In [224]:
df.shape

(5942, 16)

In [210]:
# df.to_csv("../data/raw/utmb_format/festival_des_templiers_100K_df_raw.csv", index = False, encoding = 'utf-8')

# Import the new dataset and master dataset. Clean the columns to ensure clean merging. Additional cleaning on 'Rank' and converting 'Race_Date' to datetime for sorting.

In [305]:
# df_new = pd.read_csv('../data/raw/utmb_format/festival_des_templiers_100K_df_raw.csv', encoding='utf-8')
# df_master = pd.read_csv('../data/master/master_cleaned/all_combined_races_master_2021_2025_exclude_western_states.csv', encoding='utf-8')

In [306]:
df_new.columns = df_new.columns.str.strip()
df_master.columns = df_master.columns.str.strip()

In [307]:
df_merged = pd.concat([df_master, df_new], ignore_index=True)

In [308]:
df_merged.shape

(50108, 15)

In [309]:
df_merged['Rank'] = df_merged['Rank'].replace('0', 'DNF')

In [311]:
df_merged.shape

(50108, 15)

In [235]:
df_merged['Race_Date'] = pd.to_datetime(df_merged['Race_Date'])

In [238]:
df_merged.shape

(61992, 16)

In [183]:
# df_merged.to_csv('../data/master/master_cleaned/all_combined_races_master_2021_2025_exclude_western_states.csv', index = False, encoding='utf-8')

In [254]:
df_master = pd.read_csv('../data/master/master_cleaned/all_combined_races_master_2021_2025_exclude_western_states.csv', encoding='utf-8')

In [255]:
df_master.shape

(56050, 15)

# Fixing F-UP

In [328]:
df = df.drop(columns=['Gender_Rank'], errors='ignore')

In [329]:
# Step 0: Create a unique row ID
df = df.reset_index().rename(columns={'index': 'Row_ID'})

In [330]:
# Step 1: Convert Time to timedelta
df['Time'] = pd.to_timedelta(df['Time'], errors='coerce')

# Step 2: Filter finishers with valid times
df_finishers = df[(df['Status'] == 'Finisher') & df['Time'].notna()].copy()

# Step 3: Assign Gender Rank within each race
df_finishers['Gender_Rank'] = (
    df_finishers.groupby(['Race_ID', 'Gender'])['Time'].rank(method='min')
)

# Step 4: Merge back using Row_ID to avoid duplication
df = df.drop(columns=['Gender_Rank'], errors='ignore')
df = df.merge(
    df_finishers[['Row_ID', 'Gender_Rank']],
    on='Row_ID',
    how='left'
)

# Optional: Drop Row_ID if you don't need it
df = df.drop(columns='Row_ID')


In [331]:
df.shape

(50108, 15)

In [333]:
def format_timedelta(td):
    if pd.isna(td):
        return ''
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

df['Time'] = df['Time'].apply(format_timedelta)


In [336]:
df.head(50)

,Series_ID,Race_ID,Race_Date,Year,Rank,Status,Name,Nationality,Gender,Age_Category,Time,Race_Name,Race_Loc,Race_Dist,Gender_Rank
0,10254,77199,2021-02-13,2021,31,Finisher,gryphon ketterling,USA,M,20-34,10:01:27,Black Canyon 100K,"Mayer, AZ",100K,26.0
1,10254,77199,2021-02-13,2021,32,Finisher,kalie demerjian,USA,F,20-34,10:01:32,Black Canyon 100K,"Mayer, AZ",100K,6.0
2,10254,77199,2021-02-13,2021,33,Finisher,mike mcmonagle,USA,M,20-34,10:05:17,Black Canyon 100K,"Mayer, AZ",100K,27.0
3,10254,77199,2021-02-13,2021,34,Finisher,kenneth ringled,USA,M,35-39,10:07:25,Black Canyon 100K,"Mayer, AZ",100K,28.0
4,895,89521,2021-02-13,2021,6,Finisher,jaco du plessis,NZL,M,40-44,10:07:46,Tarawera 100K,"Rotorua, New Zealand",100K,5.0
5,10254,77199,2021-02-13,2021,35,Finisher,charles macnulty,USA,M,45-49,10:09:10,Black Canyon 100K,"Mayer, AZ",100K,29.0
6,10254,77199,2021-02-13,2021,36,Finisher,phil young,USA,M,20-34,10:11:35,Black Canyon 100K,"Mayer, AZ",100K,30.0
7,10254,77199,2021-02-13,2021,37,Finisher,greg pappas,USA,M,20-34,10:11:54,Black Canyon 100K,"Mayer, AZ",100K,31.0
8,895,89521,2021-02-13,2021,7,Finisher,konoka azumi,JPN,F,20-34,10:12:09,Tarawera 100K,"Rotorua, New Zealand",100K,2.0
9,895,89521,2021-02-13,2021,8,Finisher,joe gallaher,NZL,M,20-34,10:12:26,Tarawera 100K,"Rotorua, New Zealand",100K,6.0


In [338]:
df.to_csv('../data/master/master_cleaned/all_combined_races_master_2021_2025_exclude_western_states_test_test_test.csv', index = False, encoding='utf-8')

In [339]:
df.shape

(50108, 15)

# Fixing big F-UP: Duplication on Festival des templiers 

In [291]:
race_ids_to_remove = [62321,62322,62323,62324,62325]

In [292]:
df_master_cleaned = df_master[~df_master['Race_ID'].isin(race_ids_to_remove)]

In [293]:
df_master_cleaned.shape

(44166, 15)

In [341]:
# df.to_csv('../data/master/master_cleaned/all_combined_races_master_2021_2025_exclude_western_states.csv', index = False, encoding='utf-8')